# - Data Oversampling

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
data = pd.read_csv("data/final_data.csv", index_col=0)

In [3]:
# 노년 연령대만 추출
data = data[data['연령대'] == '노년']

In [4]:
data = data.reset_index(drop=True)

In [5]:
data.columns

Index(['시간대', '성별', '연령대', '지역', '기분', '1순위', '2순위', '3순위', '기온', '강수량', '습도',
       '기압', '전운량', '불고기', '냉면', '칼국수', '된장찌개', '비빔밥', '김치찌개', '떡볶이', '김밥',
       '파스타', '스테이크', '햄버거', '초밥', '우동', '메밀소바', '돈가스', '볶음밥', '짜장면', '짬뽕',
       '날짜', '시간', '요일'],
      dtype='object')

In [6]:
data

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,기온,강수량,...,초밥,우동,메밀소바,돈가스,볶음밥,짜장면,짬뽕,날짜,시간,요일
0,2021-05-16 16:00:00,여자,노년,서울,중립,한식,일식,분식,17.7,4.8,...,0,2,0,0,0,0,0,05-16,16,일
1,2021-05-18 12:00:00,여자,노년,서울,행복,일식,분식,한식,20.5,0.0,...,3,0,0,0,0,0,0,05-18,12,화
2,2021-05-18 13:00:00,남자,노년,서울,행복,한식,중식,일식,22.2,0.0,...,1,0,0,0,0,2,0,05-18,13,화
3,2021-05-18 18:00:00,남자,노년,서울,행복,한식,일식,중식,22.6,0.0,...,2,0,0,0,0,0,1,05-18,18,화
4,2021-05-19 06:00:00,남자,노년,수도권,행복,한식,양식,일식,10.7,0.0,...,0,1,0,1,0,0,0,05-19,6,수


### oversampling 함수 정의

In [10]:
new_data = pd.DataFrame(columns=['성별', '연령대', '지역', '기분', '1순위', '2순위', '기온'])

category_list = ['한식', '중식', '일식', '분식', '양식']
category_dic = {}

tt = data['1순위'].value_counts()


print(tt)
for category in category_list:
    if category not in list(tt.index):
        category_dic[category] = 0
    else:
        pass
    

tt = tt.append(pd.Series(category_dic)) + 1 
print(tt)
ts = tt / tt.sum()
print(ts)


np.random.choice(ts.index, 42, p=ts.values)



한식    4
일식    1
Name: 1순위, dtype: int64
한식    5
일식    2
중식    1
분식    1
양식    1
dtype: int64
한식    0.5
일식    0.2
중식    0.1
분식    0.1
양식    0.1
dtype: float64


array(['한식', '한식', '중식', '일식', '양식', '한식', '일식', '일식', '중식', '분식', '한식',
       '일식', '한식', '한식', '한식', '한식', '중식', '양식', '일식', '한식', '중식', '한식',
       '한식', '분식', '양식', '일식', '한식', '한식', '한식', '한식', '일식', '한식', '한식',
       '일식', '한식', '중식', '한식', '일식', '분식', '한식', '양식', '일식'], dtype=object)

In [51]:
category_list.remove('한식')
category_list


['중식', '일식', '분식', '양식']

In [56]:
data['2순위'].value_counts()

일식    2
양식    1
중식    1
분식    1
Name: 2순위, dtype: int64

In [64]:
emotion_dic = {'우울': 0 , '분노': 0}

tmp = data['기분'].value_counts()

tmp.append(pd.Series(emotion_dic)) + 1



행복    5
중립    2
우울    1
분노    1
dtype: int64

In [282]:
def oversampling(x):
    d = data[x].value_counts() / len(data)
    print(d.index)
    print(d.values)

    random.seed(1234)
    random_data = np.random.choice(len(d.index), 42, p=d.values)
    
    random_list = [d.index[n] for n in random_data]
    return random_list

In [283]:
data['성별'].value_counts() / len(data)

남자    0.625
여자    0.375
Name: 성별, dtype: float64

In [284]:
data.loc[data['1순위']=='한식', '2순위'].value_counts()

중식    3
일식    3
양식    1
Name: 2순위, dtype: int64

In [285]:
sex = oversampling('성별')
age = oversampling('연령대')
feeling = oversampling('기분')
first = oversampling('1순위')
second = oversampling('2순위')
third = oversampling('3순위')

Index(['남자', '여자'], dtype='object')
[0.625 0.375]
Index(['노년'], dtype='object')
[1.]
Index(['행복', '중립'], dtype='object')
[0.875 0.125]
Index(['한식', '일식'], dtype='object')
[0.875 0.125]
Index(['중식', '일식', '양식', '분식'], dtype='object')
[0.375 0.375 0.125 0.125]
Index(['일식', '중식', '분식', '한식'], dtype='object')
[0.375 0.25  0.25  0.125]


In [286]:
final = pd.DataFrame([sex, age, feeling, first, second, third]).transpose()

In [287]:
final.columns = ['성별', '연령대', '기분', '1순위', '2순위', '3순위']

In [298]:
final.head()

,성별,연령대,기분,1순위,2순위,3순위
0,여자,노년,행복,한식,중식,중식
1,여자,노년,행복,한식,일식,일식
2,남자,노년,행복,일식,중식,중식
3,여자,노년,행복,한식,일식,일식
4,남자,노년,행복,한식,일식,한식


In [289]:
d = data['2순위'].value_counts() / len(data)
print(d.index)
print(d.values)

Index(['중식', '일식', '양식', '분식'], dtype='object')
[0.375 0.375 0.125 0.125]


### 각 메뉴에 1을 더하는 함수 정의

In [271]:
def plusone(col):
    cnt = data[col].value_counts()
    print(cnt)
    
    s = pd.Series() # 빈 시리즈 생성
    

In [297]:
tt = data['1순위'].value_counts()
type(tt)

pandas.core.series.Series

In [250]:
d = {'중식': 0, '양식': 0, '분식': 0}

t = pd.Series(d)
t

중식    0
양식    0
분식    0
dtype: int64

In [253]:
s = tt.append(t) + 1
s = s / s.sum()
s

한식    0.615385
일식    0.153846
중식    0.076923
양식    0.076923
분식    0.076923
dtype: float64

In [248]:
s
np.random.choice(len(s.index), 42, p=s.values)

array([2, 3, 1, 1, 3, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 3, 3, 3, 2,
       3, 0, 0, 0, 1, 0, 0, 0, 3, 0, 2, 0, 2, 0, 1, 4, 3, 0, 2, 0])